## **RAG의 핵심, 문서 검색기 Retriever**

In [1]:
!pip install -q langchain tiktoken langchain-text-splitters langchain-openai langchain-chroma pypdf

### **Retriever의 기본형, 벡터DB 기반 Retriever**

**Chroma 벡터 DB 기반 기본 유사 문서 검색**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -U langchain langchain-community

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import os
Chroma().delete_collection()
os.environ["OPENAI_API_KEY"] = "your api"

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"/content/drive/MyDrive/WB/2025JPO/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

#Chroma를 Retriever로 활용
retriever = db.as_retriever()
retriever.invoke("국회의원의 의무")

[Document(id='01b316b1-4a45-4083-93fb-aa8454bef792', metadata={'creationdate': '2024-12-06T14:47:30+09:00', 'source': '/content/drive/MyDrive/WB/2025JPO/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'moddate': '2024-12-06T14:48:14+09:00', 'page_label': '2', 'creator': 'PyPDF', 'page': 1, 'producer': 'iText 2.1.7 by 1T3XT', 'total_pages': 2}, page_content='⑤국회가 재적의원 과반수의 찬성으로 계엄의 해제를 요구한 때에는 대통령은 이를 해제하여야 한다.\n \n제89조 다음 사항은 국무회의의 심의를 거쳐야 한다.\n1. 국정의 기본계획과 정부의 일반정책\n2. 선전ㆍ강화 기타 중요한 대외정책\n3. 헌법개정안ㆍ국민투표안ㆍ조약안ㆍ법률안 및 대통령령안\n4. 예산안ㆍ결산ㆍ국유재산처분의 기본계획ㆍ국가의 부담이 될 계약 기타 재정에 관한 중요사항\n5. 대통령의 긴급명령ㆍ긴급재정경제처분 및 명령 또는 계엄과 그 해제\n6. 군사에 관한 중요사항\n7. 국회의 임시회 집회의 요구\n8. 영전수여\n9. 사면ㆍ감형과 복권\n10. 행정각부간의 권한의 획정\n11. 정부 안의 권한의 위임 또는 배정에 관한 기본계획\n12. 국정처리상황의 평가ㆍ분석\n13. 행정각부의 중요한 정책의 수립과 조정\n14. 정당해산의 제소\n15. 정부에 제출 또는 회부된 정부의 정책에 관계되는 청원의 심사'),
 Document(id='ebbcc4d0-f058-49ae-819d-834d8fe7192e', metadata={'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'total_pages': 2, 'page': 0, 'source': '/content/drive/MyDrive/WB

**(1) 유사도 점수도 함께 출력하기**

In [8]:
result_score = db.similarity_search_with_score("국회의원의 의무") #값이 낮을 수록 질문과의 유사도가 높음
result_r_score = db.similarity_search_with_relevance_scores("국회의원의 의무") #값이 높을수록 질문과 유사도가 높음

print("[유사 청크 1순위]")
print(result_score[0][0].page_content)

print("\n\n[점수]")
print(result_score[0][1])
print(result_r_score[0][1])

[유사 청크 1순위]
⑤국회가 재적의원 과반수의 찬성으로 계엄의 해제를 요구한 때에는 대통령은 이를 해제하여야 한다.
 
제89조 다음 사항은 국무회의의 심의를 거쳐야 한다.
1. 국정의 기본계획과 정부의 일반정책
2. 선전ㆍ강화 기타 중요한 대외정책
3. 헌법개정안ㆍ국민투표안ㆍ조약안ㆍ법률안 및 대통령령안
4. 예산안ㆍ결산ㆍ국유재산처분의 기본계획ㆍ국가의 부담이 될 계약 기타 재정에 관한 중요사항
5. 대통령의 긴급명령ㆍ긴급재정경제처분 및 명령 또는 계엄과 그 해제
6. 군사에 관한 중요사항
7. 국회의 임시회 집회의 요구
8. 영전수여
9. 사면ㆍ감형과 복권
10. 행정각부간의 권한의 획정
11. 정부 안의 권한의 위임 또는 배정에 관한 기본계획
12. 국정처리상황의 평가ㆍ분석
13. 행정각부의 중요한 정책의 수립과 조정
14. 정당해산의 제소
15. 정부에 제출 또는 회부된 정부의 정책에 관계되는 청원의 심사


[점수]
1.009122371673584
0.286573383172941


**(2) 검색 결과 수 및 조정**

In [9]:
#유사 청크 1개만 반환
retriever = db.as_retriever(search_kwargs={"k": 1}) #매개 변수를 활용, 더 세밀한 검색을 실시하기 "K" 를 조정하기
retriever.invoke("국회의원의 의무")

[Document(id='01b316b1-4a45-4083-93fb-aa8454bef792', metadata={'page_label': '2', 'creator': 'PyPDF', 'creationdate': '2024-12-06T14:47:30+09:00', 'producer': 'iText 2.1.7 by 1T3XT', 'source': '/content/drive/MyDrive/WB/2025JPO/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'total_pages': 2, 'page': 1, 'moddate': '2024-12-06T14:48:14+09:00'}, page_content='⑤국회가 재적의원 과반수의 찬성으로 계엄의 해제를 요구한 때에는 대통령은 이를 해제하여야 한다.\n \n제89조 다음 사항은 국무회의의 심의를 거쳐야 한다.\n1. 국정의 기본계획과 정부의 일반정책\n2. 선전ㆍ강화 기타 중요한 대외정책\n3. 헌법개정안ㆍ국민투표안ㆍ조약안ㆍ법률안 및 대통령령안\n4. 예산안ㆍ결산ㆍ국유재산처분의 기본계획ㆍ국가의 부담이 될 계약 기타 재정에 관한 중요사항\n5. 대통령의 긴급명령ㆍ긴급재정경제처분 및 명령 또는 계엄과 그 해제\n6. 군사에 관한 중요사항\n7. 국회의 임시회 집회의 요구\n8. 영전수여\n9. 사면ㆍ감형과 복권\n10. 행정각부간의 권한의 획정\n11. 정부 안의 권한의 위임 또는 배정에 관한 기본계획\n12. 국정처리상황의 평가ㆍ분석\n13. 행정각부의 중요한 정책의 수립과 조정\n14. 정당해산의 제소\n15. 정부에 제출 또는 회부된 정부의 정책에 관계되는 청원의 심사')]

**(3) 검색 방식 변경 - MMR**

In [11]:
#유사도를 기준으로, 높은 유사도 순의 청크 N개만 추출 (MMR - Maximal Marginal Relevance)
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"/content/drive/MyDrive/WB/2025JPO/대한민국헌법(헌법)(제00010호)(19880225).pdf")

pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

In [12]:
#Chroma를 Retriever로 활용 - 다양성만이 고려된 경우
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs = {"lambda_mult": 0, "fetch_k":10, "k":3}
)
retriever.invoke("국회의원의 의무")

[Document(id='01b316b1-4a45-4083-93fb-aa8454bef792', metadata={'page_label': '2', 'page': 1, 'total_pages': 2, 'creationdate': '2024-12-06T14:47:30+09:00', 'producer': 'iText 2.1.7 by 1T3XT', 'moddate': '2024-12-06T14:48:14+09:00', 'creator': 'PyPDF', 'source': '/content/drive/MyDrive/WB/2025JPO/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='⑤국회가 재적의원 과반수의 찬성으로 계엄의 해제를 요구한 때에는 대통령은 이를 해제하여야 한다.\n \n제89조 다음 사항은 국무회의의 심의를 거쳐야 한다.\n1. 국정의 기본계획과 정부의 일반정책\n2. 선전ㆍ강화 기타 중요한 대외정책\n3. 헌법개정안ㆍ국민투표안ㆍ조약안ㆍ법률안 및 대통령령안\n4. 예산안ㆍ결산ㆍ국유재산처분의 기본계획ㆍ국가의 부담이 될 계약 기타 재정에 관한 중요사항\n5. 대통령의 긴급명령ㆍ긴급재정경제처분 및 명령 또는 계엄과 그 해제\n6. 군사에 관한 중요사항\n7. 국회의 임시회 집회의 요구\n8. 영전수여\n9. 사면ㆍ감형과 복권\n10. 행정각부간의 권한의 획정\n11. 정부 안의 권한의 위임 또는 배정에 관한 기본계획\n12. 국정처리상황의 평가ㆍ분석\n13. 행정각부의 중요한 정책의 수립과 조정\n14. 정당해산의 제소\n15. 정부에 제출 또는 회부된 정부의 정책에 관계되는 청원의 심사'),
 Document(id='8209906a-6195-4bab-a026-f3bd5fe09d21', metadata={'page_label': '2', 'producer': 'iText 2.1.7 by 1T3XT', 'creationdate': '2024-12-06T14:47:30+09:00', 'total_pages': 2, 'pa

**일반 유사도 검색 방식**

In [13]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(search_kwargs = {"k":3})
retriever.get_relevant_documents("국회의원의 의무")


/tmp/ipython-input-13-2706477368.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("국회의원의 의무")


[Document(id='01b316b1-4a45-4083-93fb-aa8454bef792', metadata={'producer': 'iText 2.1.7 by 1T3XT', 'source': '/content/drive/MyDrive/WB/2025JPO/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'creator': 'PyPDF', 'total_pages': 2, 'creationdate': '2024-12-06T14:47:30+09:00', 'page_label': '2', 'moddate': '2024-12-06T14:48:14+09:00', 'page': 1}, page_content='⑤국회가 재적의원 과반수의 찬성으로 계엄의 해제를 요구한 때에는 대통령은 이를 해제하여야 한다.\n \n제89조 다음 사항은 국무회의의 심의를 거쳐야 한다.\n1. 국정의 기본계획과 정부의 일반정책\n2. 선전ㆍ강화 기타 중요한 대외정책\n3. 헌법개정안ㆍ국민투표안ㆍ조약안ㆍ법률안 및 대통령령안\n4. 예산안ㆍ결산ㆍ국유재산처분의 기본계획ㆍ국가의 부담이 될 계약 기타 재정에 관한 중요사항\n5. 대통령의 긴급명령ㆍ긴급재정경제처분 및 명령 또는 계엄과 그 해제\n6. 군사에 관한 중요사항\n7. 국회의 임시회 집회의 요구\n8. 영전수여\n9. 사면ㆍ감형과 복권\n10. 행정각부간의 권한의 획정\n11. 정부 안의 권한의 위임 또는 배정에 관한 기본계획\n12. 국정처리상황의 평가ㆍ분석\n13. 행정각부의 중요한 정책의 수립과 조정\n14. 정당해산의 제소\n15. 정부에 제출 또는 회부된 정부의 정책에 관계되는 청원의 심사'),
 Document(id='c65dffc6-b9cd-4e18-af16-1cafb6b63fd2', metadata={'moddate': '2024-12-06T14:48:14+09:00', 'total_pages': 2, 'producer': 'iText 2.1.7 by 1T3XT', 'creationdate': '2024-12-06

### **사용자의 쿼리를 재해석하여 검색하다, MultiQueryRetriever**

**Chroma DB에 문서 벡터 저장**

In [15]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"/content/drive/MyDrive/WB/2025JPO/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

**질문을 여러 버전으로 재해석하여 Retriever에 활용**

In [17]:
# 대한민국 헌법 PDF를 Chroma DB로 임베딩/저장한 것은 위 셀에서 완료됨

# 여러 개의 유사 질문을 생성하기 위한 모듈 및 모델 선언
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# 검색에 사용할 질문 입력
question = "국회의원의 의무는 무엇이 있나요?"

# LLM 설정 (OpenAI의 GPT-4o-mini 사용, temperature는 0으로 고정)
# temperature=0은 가능한 한 일관된 출력(다양성보다 정답성 중시)을 의미
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Chroma 벡터DB에서 검색하는 Retriever 객체 + LLM 기반 MultiQueryRetriever 설정
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(),  # 기존 Chroma DB에서 의미 기반 검색을 담당
    llm=llm                        # 다양한 표현으로 쿼리를 확장할 LLM
)

# 검색 쿼리 생성 및 과정 로깅을 위해 로깅 수준 설정
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# 위 질문을 다양한 표현으로 확장하고, 각 쿼리로 유사한 문서 검색
unique_docs = retriever_from_llm.invoke(input=question)

# 검색 결과로 반환된 고유 문서 개수 출력
len(unique_docs)


INFO:langchain.retrievers.multi_query:Generated queries: ['국회의원이 수행해야 하는 의무는 어떤 것들이 있나요?  ', '국회의원의 책임과 의무에 대해 설명해 주세요.  ', '국회의원으로서 지켜야 할 의무는 무엇인가요?']


6

### **문서를 여러 벡터로 재해석하다, MultiVectorRetriever**

**Chroma DB에 문서 벡터 저장**

In [18]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"/content/drive/MyDrive/WB/2025JPO/대한민국헌법(헌법)(제00010호)(19880225).pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
docs = text_splitter.split_documents(docs)

**Multi Vector를 만들기 위한 작업 (상위 검색기를 구축하기 위함)**

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
 ) #huggingface 임베딩 된 것 확인

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embedding
)
# 상위 문서 저장 위한 레이어 선언 (상위 청크와 하위 청크를 연결하여 저장할 저장소 역할, docs에 저장하기)
store = InMemoryByteStore()
id_key = "doc_id"
# 상위 문서와 하위 문서를 연결할 키값으로 doc_id 사용
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

#문서 id로 고유한 값을 지정하기 위해 uuid 라이브러리 호출
import uuid
doc_ids = [str(uuid.uuid4()) for _ in docs]

/tmp/ipython-input-19-179998917.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
# 하위 청크로 쪼개기 위한 child_text_splitter 지정
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# 상위 청크들을 순회하며 하위 청크로 분할한 후 상위 청크 id 상속
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)
#vectorstore에 하위 청크 추가
retriever.vectorstore.add_documents(sub_docs)
#docstore에 상위청크 저장할 때, doc_ids 지정
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [21]:
# Vectorstore alone retrieves the small chunks
print("[하위 청크] \n")
print(retriever.vectorstore.similarity_search("국민의 권리")[0].page_content)
print("-"*50)
print("[상위 청크] \n")
print(retriever.invoke("국민의 권리")[0].page_content)

[하위 청크] 

제1장 총강
 
제1조 ①대한민국은 민주공화국이다.
②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.
 
제5조 ①대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.
②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.
 
제7조 ①공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.
②공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 보장된다.
 
제8조 ①정당의 설립은 자유이며, 복수정당제는 보장된다.
②정당은 그 목적ㆍ조직과 활동이 민주적이어야 하며, 국민의 정치적 의사형성에 참여하는데 필요한 조직을 가져야
한다.
--------------------------------------------------
[상위 청크] 

법제처                                                            1                                                       국가법령정보센터
대한민국헌법
 
대한민국헌법
[시행 1988. 2. 25.] [헌법 제10호, 1987. 10. 29., 전부개정]
       제1장 총강
 
제1조 ①대한민국은 민주공화국이다.
②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.
 
제5조 ①대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.
②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.
 
제7조 ①공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.
②공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 보장된다.
 
제8조 ①정당의 설립은 자유이며, 복수정당제는 보장된다.
②정당은 그 목적ㆍ조직과 활동이 민주적이어야 하며, 국민의 정치적 의사형성에 참여하는데 필요한 조직을 가져야
한다.
③정당은 법률이 정하는 바에 의하여 국가의

### **컨텍스트 재정렬, Long-Context Reorder**

**[Long-Context Reorder 없이 유사 문서 출력]**

In [ ]:
#Chroma dimension 관련 에러 발생 시 실행
# Chroma().delete_collection()

In [22]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_chroma import Chroma
from langchain.document_transformers import (
    LongContextReorder,
)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

Chroma().delete_collection()

# 한글 임베딩 모델 선언
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다."
    "저는 단순히 영화 보러 가는 것을 좋아합니다",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적인 NBA 선수였습니다.",
]
# Chroma Retriever 선언(10개의 유사 문서 출력)
retriever = Chroma.from_texts(texts, embedding=embedding).as_retriever(
    search_kwargs={"k": 10}
)
query = "셀틱에 대해 설명해줘"

# 유사도 기준으로 검색 결과 출력
docs = retriever.invoke(query)
docs

[Document(id='efde1968-f9c2-49b2-8d55-58003ef5b87b', metadata={}, page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(id='da67b929-aec5-4141-a892-f46dc1d66e10', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(id='b5fae81f-c704-4e93-93c3-334596f93add', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='62d4e6ef-d396-42c9-8120-c44d2b6b647b', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(id='11e97753-abe5-482c-bc0b-9a7ebef6909d', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='2de97df6-871c-4362-8bec-b3eacad36b63', metadata={}, page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(id='a884db73-e71a-484f-834f-c67182116092', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='fa83298d-25aa-478a-87e2-cf5b55314b63', metadata={}, page_content='엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.'),
 Document(id='35ce6ee5-5017-405d-8f79-0a614689be57', metadata={}, page_content='래리 버드는 상징적인 NBA 선수였습니다.')]

**[Long-Context Reorder 활용하여 유사 문서 출력]**

In [23]:
#LongContextReorder 선언
reordering = LongContextReorder()
#검색된 유사문서 중 관련도가 높은 문서를 맨앞과 맨뒤에 재정배치
reordered_docs = reordering.transform_documents(docs)
reordered_docs

[Document(id='efde1968-f9c2-49b2-8d55-58003ef5b87b', metadata={}, page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(id='b5fae81f-c704-4e93-93c3-334596f93add', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='11e97753-abe5-482c-bc0b-9a7ebef6909d', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='a884db73-e71a-484f-834f-c67182116092', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='35ce6ee5-5017-405d-8f79-0a614689be57', metadata={}, page_content='래리 버드는 상징적인 NBA 선수였습니다.'),
 Document(id='fa83298d-25aa-478a-87e2-cf5b55314b63', metadata={}, page_content='엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.'),
 Document(id='2de97df6-871c-4362-8bec-b3eacad36b63', metadata={}, page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(id='62d4e6ef-d396-42c9-8120-c44d2b6b647b', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(id='da67b929-aec5-4141-a892-f46dc1d66e10', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.')]